# Backtes the three PTS Methods

In [1]:
import pandas as pd

from Methods.DmHelpers import *
from Methods.CointHelpers import *
from Methods.CopulasHelpers import *

from Helpers.DataHelpers import *
from Helpers.PlotHelpers import *
from Helpers.ModuleHelpers import *
from Helpers.BacktestHelpers import *

In [2]:
# Donwnload data
tickers = pd.read_excel('Data/SandP500list.xlsx')
tickers = tickers[tickers['sector'] == 'Financials']
names = tickers['symbol'].tolist()

START_DATE = pd.to_datetime('2020-01-01') # Every year takes approximately 30 seconds for coint and dm
END_DATE = pd.to_datetime('2024-04-01') # test 6 months yyy

data = download_data(names, START_DATE, END_DATE)

[*********************100%***********************]  65 of 65 completed


6 Failed downloads:
['PBCT', 'BRK.B', 'FRC', 'WLTW', 'RE']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SIVB']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


In [3]:
# Run rolling window overlapping backtest

start_date, end_date = START_DATE, START_DATE + pd.DateOffset(months=18)

dm_backtest_results = pd.DataFrame()
coint_backtest_results = pd.DataFrame()
copula_backtest_results = pd.DataFrame()

counter = 0

while end_date < END_DATE:
        # Split data into training and testing
    train_start, train_end, test_start, test_end = train_test_dates(start_date)
    train_data, test_data = train_test_split(data, train_start, train_end, test_start, test_end)
        
    # Run backtests for each strategy
    dm_signals, dm_pairs = dm_get_signals_backtest(train_data, test_data, threshold=2) # was 1.5
    dm_signals_test = dm_signals.loc[test_start:test_end]
    coint_signals, coint_pairs = coint_get_signals_backtest(train_data, test_data, threshold=2) 
    coint_signals_test = coint_signals.loc[test_start:test_end]
    copula_signals, copula_pairs = copula_get_signals_backtest(train_data, test_data, threshold=0.8)
    copula_signals_test = copula_signals.loc[test_start:test_end]

    # Helper function to process each set of signals
    def process_signals(signals, results_df, label):
        tuples = [(label, col) for col in signals.columns]
        multi_index = pd.MultiIndex.from_tuples(tuples, names=['Period', 'Signal'])
        period_signals = pd.DataFrame(signals.values, index=signals.index, columns=multi_index)
        return pd.concat([results_df, period_signals], axis=1)

    # Format the period label for the big column
    period_label = f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"

        # Process each method's signals and update the respective DataFrame
    dm_backtest_results = process_signals(dm_signals_test, dm_backtest_results, period_label)
    coint_backtest_results = process_signals(coint_signals_test, coint_backtest_results, period_label)
    copula_backtest_results = process_signals(copula_signals_test, copula_backtest_results, period_label)

        # Update for next period
    print_backtest_info(counter, start_date, end_date)
    start_date, end_date = update_dates(start_date)
    counter += 1


Backtest Period: 1
Start Date: 2020-01-01 End Date: 2021-07-01
-------------------------------------------
Backtest Period: 2
Start Date: 2020-02-01 End Date: 2021-08-01
-------------------------------------------
Backtest Period: 3
Start Date: 2020-03-01 End Date: 2021-09-01
-------------------------------------------
Backtest Period: 4
Start Date: 2020-04-01 End Date: 2021-10-01
-------------------------------------------
Backtest Period: 5
Start Date: 2020-05-01 End Date: 2021-11-01
-------------------------------------------
Backtest Period: 6
Start Date: 2020-06-01 End Date: 2021-12-01
-------------------------------------------
Backtest Period: 7
Start Date: 2020-07-01 End Date: 2022-01-01
-------------------------------------------
Backtest Period: 8
Start Date: 2020-08-01 End Date: 2022-02-01
-------------------------------------------
Backtest Period: 9
Start Date: 2020-09-01 End Date: 2022-03-01
-------------------------------------------
Backtest Period: 10
Start Date: 2020-

/Users/lasse/Desktop/EBSUni/Msc_Semester_1/Empirical Finance/PairsTradingAll/Pairs-Trading-Strategy/Code/Methods/CopulasHelpers.py:262: RuntimeWarning: invalid value encountered in divide
  conditional_cdf = t.cdf(numerator / denominator, df=nu + 1)


Backtest Period: 15
Start Date: 2021-03-01 End Date: 2022-09-01
-------------------------------------------
Backtest Period: 16
Start Date: 2021-04-01 End Date: 2022-10-01
-------------------------------------------
Backtest Period: 17
Start Date: 2021-05-01 End Date: 2022-11-01
-------------------------------------------
Backtest Period: 18
Start Date: 2021-06-01 End Date: 2022-12-01
-------------------------------------------
Backtest Period: 19
Start Date: 2021-07-01 End Date: 2023-01-01
-------------------------------------------
Backtest Period: 20
Start Date: 2021-08-01 End Date: 2023-02-01
-------------------------------------------
Backtest Period: 21
Start Date: 2021-09-01 End Date: 2023-03-01
-------------------------------------------
Backtest Period: 22
Start Date: 2021-10-01 End Date: 2023-04-01
-------------------------------------------
Backtest Period: 23
Start Date: 2021-11-01 End Date: 2023-05-01
-------------------------------------------
Backtest Period: 24
Start Da

In [4]:
# get cumulative returns for each method
dm_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(dm_backtest_results, data))
coint_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(coint_backtest_results, data)).fillna(0)
coint_average_returns_tcosts = coint_average_returns_tcosts.loc[pd.to_datetime('2021-01-01'):]
copula_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(copula_backtest_results, data))

dm_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(dm_backtest_results, data))
coint_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(coint_backtest_results, data)).fillna(0)
coint_average_returns_no_tcosts = coint_average_returns_no_tcosts.loc[pd.to_datetime('2021-01-01'):]
copula_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(copula_backtest_results, data))

# Combine all dfs and save to excel
all_dfs = [dm_average_returns_tcosts, coint_average_returns_tcosts, copula_average_returns_tcosts, dm_average_returns_no_tcosts, coint_average_returns_no_tcosts, copula_average_returns_no_tcosts]
all_dfs = pd.concat(all_dfs, axis=1)
all_dfs.columns = ['dm_average_returns_tcosts', 'coint_average_returns_tcosts', 'copula_average_returns_tcosts', 'dm_average_returns_no_tcosts', 'coint_average_returns_no_tcosts', 'copula_average_returns_no_tcosts']
all_dfs.to_excel('Data/all_average_returns.xlsx')